# Google MAP 方法

In [1]:
import googlemaps
import dotenv
import sqlite3
import pandas as pd

In [2]:
info = dotenv.dotenv_values('.env')
apikey = info.get('googleAPI')
gmaps = googlemaps.Client(key=apikey)

# mode : driving（車）、transit（公車之類）、walking（步行）、無機車模式
# google是算交通距離、還可以有時間
def GM(origin, destination, mode = 'driving'):
    response = gmaps.distance_matrix(origin, destination, mode=mode)
    try:
        distance = response['rows'][0]['elements'][0]['distance']['value'] # return int
        duration = response['rows'][0]['elements'][0]['duration']['text'] # return string
        return distance, duration
    except:
        return None, None

In [3]:
# 連線資料庫、SQL搜尋、轉df
conn = sqlite3.connect('TW_cinema_info.db')
data_df = pd.read_sql_query("SELECT * FROM TW_cinema_info_latest", conn)
conn.close()

In [4]:
# 找使用者座標附近6間戲院
def MathJudge_dist(df, location):
    df['緯度'] = df['經緯度'].map(lambda x:float(str(x).strip('[]').split(',')[0])) - float(location.split(',')[0])
    df['經度'] = df['經緯度'].map(lambda x:float(str(x).strip('[]').split(',')[1])) - float(location.split(',')[1])

    # 排序直線距離短的 (這是用經緯度數字直接去算，不是實際有效交通距離)
    # 至少能過濾掉，站在宜蘭但推我同緯度的新竹，卻沒推到雙北基隆這類問題
    df['直線距離'] = (df['緯度'] ** 2 + df['經度'] ** 2) ** 0.5
    df = df.drop(['緯度', '經度'], axis = 1)
    df = df.sort_values(['直線距離']).reset_index(drop=True)

    # 取得前6筆
    return df.head(6)
    

# 算這6間距離、時間、模式(可不選，預設開車)
def order_df_dist_google(df, origin, mode = 'driving'):
    df = df.copy()
    df['temp'] = df['經緯度'].map(lambda x: GM(origin, str(x).strip('[]'), mode))
    df['距離(公尺)'] = df['temp'].apply(lambda x:x[0])
    df['預計時間'] = df['temp'].apply(lambda x:x[1])
    df.drop('temp', axis = 1, inplace = True)
    df = df.sort_values('距離(公尺)').reset_index(drop=True)

    # 挑出前3個要返回的資訊、有Null、NAN代表無路線可到該戲院
    res = df[['事業名稱', '距離(公尺)', '預計時間']].dropna().head(3)

    return res

In [13]:
location = '24.171550, 120.645647'
location = '23.924534, 121.564256'
location = '25.037741732578425,121.50864867701743'
# location = '23.566450, 119.624411' # 澎湖
location = '24.464436, 118.440796' # 金門
the_nearest_cinema = MathJudge_dist(data_df, location)

In [14]:
the_nearest_cinema

,縣市,事業名稱,地址,經緯度,直線距離
0,金門縣,國賓影城@金門昇恆昌金湖廣場,金門縣金湖鎮太湖路二段198號6樓,"[24.4392162, 118.4186129]",0.033588
1,金門縣,金獅影城,金門縣金湖鎮中山路8-5號西棟3樓,"[24.4417138, 118.3630416]",0.081006
2,澎湖縣,澎湖in89豪華影城,澎湖縣馬公市同和路158號4樓,"[23.56734, 119.572906]",1.444456
3,雲林縣,北港秀泰影城,雲林縣北港鎮光復里華南路101號2樓,"[23.5958633, 120.2858846]",2.039306
4,雲林縣,白宮戲院,雲林縣虎尾鎮中正路257號,"[23.705542, 120.431131]",2.130106
5,彰化縣,台灣大戲院,彰化縣彰化市中正路二段48號,"[24.0806893, 120.5382102]",2.132231


In [15]:
mode = 'walking'
order_df_dist_google(the_nearest_cinema, location)

,事業名稱,距離(公尺),預計時間
0,國賓影城@金門昇恆昌金湖廣場,4953.0,10 mins
1,金獅影城,11062.0,17 mins


# 直線距離算法

In [16]:
# 這套是算直線距離, return float
from Function_list import distance_m
def order_df_dist_GC(df, origin):
    df = df.copy()
    df['距離(公尺)'] = df['經緯度'].map(lambda x: int(distance_m(origin, eval(x))))
    df = df.sort_values('距離(公尺)').reset_index(drop=True)
    res = df[['事業名稱', '距離(公尺)']].head(3)
    return res

In [17]:
order_df_dist_GC(the_nearest_cinema, location)

,事業名稱,距離(公尺)
0,國賓影城@金門昇恆昌金湖廣場,3586
1,金獅影城,8275
2,澎湖in89豪華影城,152113
